In [6]:
import csv
import tqdm
import numpy as np
import matplotlib.pyplot as plt
from string import digits

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset

seed = 42
epochs = 20
batch_size = 32
learning_rate = 1e-3
context_frames = 10
sequence_length = 20
lookback = sequence_length

context_epochs = 20
context_batch_size = 1
context_learning_rate = 1e-3
context_data_length = 20

trainsize = 0.8
valsize = 0.9

torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")#  use gpu if available

In [2]:
class FullDataSet():
    def __init__(self, chopmin, chopmax, testset=False, dataset=False):
        if not testset:
            dataset_full = []
            for value in data_map[1:]:  # ignore header
                dataset_full.append(self.forward(value))
            self.samples = dataset_full[int(len(dataset_full)*chopmin):int(len(dataset_full)*chopmax)]
        if dataset:
            self.samples = dataset
                                    
    def testsetreturn(self):
        experiment = []
        dataset_full = []
        current_experiment = data_map[1][-2]
        for value in data_map[1:]:  # ignore header
            if current_experiment == value[-2]:
                experiment.append(self.forward(value))
            else:
                dataset_full.append(experiment)
                experiment = [self.forward(value)]
                current_experiment = value[-2]
        return dataset_full

    def forward(self, value):
        state = np.load(data_dir + value[4])
        return [np.load(data_dir + value[8]),
                             np.float32(np.load(data_dir + value[2])),
                             np.float32(np.load(data_dir + value[3])),
                             np.float32(np.asarray([state[0] for i in range(0, len(state))]))]

    def __len__(self):
        return len(self.samples)

    def __getitem__(self,idx):
        return(self.samples[idx])

data_dir = '/home/user/Robotics/Data_sets/slip_detection/vector_normalised_002/'
with open(data_dir + 'map.csv', 'r') as f:  # rb
    data_map = [row for row in csv.reader(f)]

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
train_loader = torch.utils.data.DataLoader(FullDataSet(0, trainsize), batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(FullDataSet(trainsize, valsize), batch_size=batch_size, shuffle=True)
test_experiments = [torch.utils.data.DataLoader(test_exp, batch_size=batch_size, shuffle=False) for test_exp in FullDataSet(0, trainsize, testset=True).testsetreturn()]
print("done")

In [12]:
class FullModel(nn.Module):
    def __init__(self):
        super(FullModel, self).__init__()
        self.lstm1 = nn.LSTM(48, 48).to(device)  # tactile
        self.lstm2 = nn.LSTM(48, 48).to(device)  # context
        self.fc1 = nn.Linear(96, 48)  # tactile + context
        self.lstm3 = nn.LSTM(6, 6).to(device)  # pos_vel
        self.fc2 = nn.Linear(54, 48)  # tactile + pos_vel
        self.lstm4 = nn.LSTM(48, 48).to(device)  # tactile, context, robot

    def forward(self, tactiles, actions, context):
        state = actions[0]
        state.to(device)
        batch_size__ = tactiles.shape[1]
        outputs = []
        hidden1 = (torch.rand(1,batch_size__,48).to(device), torch.rand(1,batch_size__,48).to(device))
        hidden2 = (torch.rand(1,batch_size__,48).to(device), torch.rand(1,batch_size__,48).to(device))
        hidden3 = (torch.rand(1,batch_size__,6).to(device), torch.rand(1,batch_size__,6).to(device))
        hidden4 = (torch.rand(1,batch_size__,48).to(device), torch.rand(1,batch_size__,48).to(device))
        for index, [sample_tactile, sample_action, sample_context] in enumerate(zip(tactiles.squeeze(), actions.squeeze(), context.squeeze())):
            # 2. Run through lstm:
            if index > context_frames-1:
                out1, hidden1 = self.lstm1(out6, hidden1)
                out2, hidden2 = self.lstm2(sample_context.unsqueeze(0).to(device), hidden2)
                context_and_tactile = torch.cat((out2.squeeze(), out1.squeeze()), 1)
                out3 = self.fc1(context_and_tactile.unsqueeze(0).cpu().detach())

                out4, hidden3 = self.lstm3(sample_action.unsqueeze(0).to(device), hidden3)
                context_and_tactile_and_robot = torch.cat((out3.squeeze().to(device), out4.squeeze()), 1)

                out5 = self.fc2(context_and_tactile_and_robot.unsqueeze(0).cpu().detach())
                out6, hidden4 = self.lstm4(out5.to(device), hidden4)
                outputs.append(out6.squeeze())
            else:
                out1, hidden1 = self.lstm1(sample_tactile.unsqueeze(0).to(device), hidden1)
                out2, hidden2 = self.lstm2(sample_context.unsqueeze(0).to(device), hidden2)
                context_and_tactile = torch.cat((out2.squeeze(), out1.squeeze()), 1)
                out3 = self.fc1(context_and_tactile.unsqueeze(0).cpu().detach())

                out4, hidden3 = self.lstm3(sample_action.unsqueeze(0).to(device), hidden3)
                context_and_tactile_and_robot = torch.cat((out3.squeeze().to(device), out4.squeeze()), 1)

                out5 = self.fc2(context_and_tactile_and_robot.unsqueeze(0).cpu().detach())
                out6, hidden4 = self.lstm4(out5.to(device), hidden4)

        return torch.stack(outputs)

In [13]:
### Train the model:
full_model = FullModel()
criterion = nn.L1Loss()
optimizer = optim.Adam(full_model.parameters(), lr=learning_rate)

early_stop_clock = 0
plot_training_loss = [1]
plot_validation_loss = [1]
progress_bar = tqdm.tqdm(range(epochs), total=(epochs*len(train_loader)))
for epoch in progress_bar:
    train_losses = 0
    val_losses = 0.0
    for index, batch_features in enumerate(train_loader):
        # 1. Reshape data and send to device:
        tactile_predictions = full_model.forward(batch_features[1].permute(1,0,2).to(device),
                                                 batch_features[2].permute(1,0,2).to(device),
                                                 batch_features[3].permute(1,0,2).to(device))
        optimizer.zero_grad()
        loss = criterion(tactile_predictions.to(device), tactile[context_frames:])
        loss.backward()
        optimizer.step()
        train_losses += loss.item()
        progress_bar.set_description("epoch: {}, ".format(epoch) + "loss: {:.4f}, ".format(float(loss.item())) + "mean loss: {:.4f}, ".format(train_losses/(index+1)))
        progress_bar.update()
    mean = train_losses / index + 1
    plot_training_loss.append(mean)

    with torch.no_grad():
        for index, batch_features in enumerate(valid_loader):
            tactile_predictions = full_model.forward(batch_features[1].permute(1,0,2).to(device),
                                                     batch_features[2].permute(1,0,2).to(device),
                                                     batch_features[3].permute(1,0,2).to(device))
            optimizer.zero_grad()
            val_loss = criterion(tactile_predictions.to(device), batch_features[1].permute(1,0,2)[context_frames:])
            val_losses += val_loss.item()

    plot_validation_loss.append(val_losses / index)

    if plot_validation_loss[-2] < plot_validation_loss[-1]:
        early_stop_clock +=1
        if early_stop_clock == 3:
            break
    else:
        early_stop_clock = 0

plt.plot(plot_training_loss[1:], c="r", label="train loss MAE")
plt.plot(plot_validation_loss[1:], c='b', label="val loss MAE")
plt.legend(loc="upper right")


  0%|          | 0/12740 [00:00<?, ?it/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 48, got 6

In [ ]:
### Test the model:
criterion1 = nn.L1Loss()
criterion2 = nn.MSELoss()
tactile_predictions_full, tactile_groundtruth_full = [], []
with torch.no_grad():
    for test_number, test_loader in enumerate(DataSampleSet):
        tactile_predictions, tactile_groundtruth = [], []
        test_lossesMAE, test_lossesMSE = 0.0, 0.0
        for index__, batch_features in enumerate(test_loader):
            tp = model.forward(batch_features.permute(0,2,1,3).to(device))
            tactile_predictions.append(tp)
            tactile_groundtruth.append(batch_features[1].permute(1,0,2)[context_frames:])
            # calculate losses
            test_lossesMAE += criterion1(tp.to(device), batch_features[1].permute(1,0,2)[context_frames:]).item()
            test_lossesMSE += criterion2(tp.to(device), batch_features[1].permute(1,0,2)[context_frames:]).item()

        print("test " + str(test_number) + " loss MAE(L1): ", str(test_lossesMAE / index__))
        print("test " + str(test_number) + " loss MSE: ", str(test_lossesMSE / index__))

        tactile_predictions_full.append(tactile_predictions)
        tactile_groundtruth_full.append(tactile_groundtruth)
print("done")

In [ ]:
# calculate tactile values for full sample:
# calculate tactile values for full sample:
predicted_data_t1, predicted_data_t9, groundtruth_data_t1 = [], [], []
tactile_predictions = tactile_predictions_full[experiment_to_test]
tactile_groundtruth = tactile_groundtruth_full[experiment_to_test]
for index, batch_set in enumerate(tactile_predictions):
    for batch in range(0, len(batch_set[0])):
        predicted_data_t1.append(batch_set[0][batch])
        groundtruth_data_t1.append(tactile_groundtruth[index][0][batch])
        predicted_data_t9.append(batch_set[5][batch])